In [ ]:
import networkx as nx
import math
import pandas as pd
import matplotlib.pyplot as plt

# Mygraph_style_base
node_color = "gray"
node_size = 600
edge_color = "black"
font_size = 15
font_color = "white"
width = 2


In [ ]:
def graph_maker(Gen_xy, graph_edges, k_num, g_kind):
    G = nx.Graph()
    G.add_edges_from(graph_edges)
    for node, coords in Gen_xy.items():
        G.add_node(node, pos=coords)
    pos = nx.get_node_attributes(G, "pos")
    nx.draw(
        G,
        pos,
        with_labels=False,
        node_color="black",
        edge_color=edge_color,
        node_size=node_size + 100,
        font_size=font_size,
        font_color=font_color,
        width=width,
    )

    plt.axis("equal")
    plt.show()


In [ ]:
#gridPebbleGame

# (2,3)-MTG(P)を生成する関数
def kl_MTG_generate(Gen_xy, k=2, l=3):  # noqa: E741 pebblegameアルゴリズム
    if (0 <= l <= 2 * k - 1) and (1 <= k):
        h = math.ceil(((2 * k + 1) + ((2 * k + 1) ** 2 - 8 * (l - 1)) ** 0.5) / 2) - 3
        graph_edges = kGG_generate(Gen_xy, h)
        edge_dis_dict = {}
        for i in graph_edges:
            x_dis = abs(Gen_xy[i[0]][0] - Gen_xy[i[1]][0])
            y_dis = abs(Gen_xy[i[0]][1] - Gen_xy[i[1]][1])
            dis = math.sqrt(x_dis**2 + y_dis**2)
            edge_dis_dict[i] = dis
        dis_edge_list = []
        dis_edge_list = sorted(edge_dis_dict.items(), key=lambda x: x[1])
        dis_edge_list = [edge[0] for edge in dis_edge_list]
        # print(edge_dis_dict)
        # print(dis_edge_list)
        # k個のpebbleを各頂点に配置
        pebble_dict = {}
        direct_edge = set()
        n = len(Gen_xy)
        G = nx.DiGraph()
        G.add_nodes_from(Gen_xy.keys())

        for i in range(n):
            pebble_dict[i] = k

        index = 0

        while True:
            # print("---")
            flag = True
            # print(direct_edge, (k * n - l))
            # print(index,len(dis_edge_list))
            # print(dis_edge_list)
            # print(pebble_dict)

            if len(direct_edge) == (k * n - l):
                # print("完了")
                graph_edges = set()
                for i in direct_edge:
                    graph_edges.add(i)
                graph_maker(
                    Gen_xy,
                    graph_edges,
                    f"({k},{l})",
                    "MTG",                                                                                                                                                                                                                             　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
                )
                return graph_edges
            u_peb_num = pebble_dict[dis_edge_list[index][0]]
            v_peb_num = pebble_dict[dis_edge_list[index][1]]
            if (u_peb_num + v_peb_num) >= (l + 1):
                if u_peb_num > v_peb_num:
                    u, v = 0, 1
                else:
                    u, v = 1, 0
                direct_edge_add = (dis_edge_list[index][u], dis_edge_list[index][v])
                direct_edge.add(direct_edge_add)
                pebble_dict[dis_edge_list[index][u]] += -1
                G.add_edge(dis_edge_list[index][u], dis_edge_list[index][v])
                index += 1
                pre_direct_edge = direct_edge.copy()
                pre_pebble_dict = pebble_dict.copy()
                pre_G = G.copy()

            else:
                dfs_edges = list(nx.dfs_edges(G, source=dis_edge_list[index][0]))

                for i in range(len(dfs_edges)):
                    if (
                        (pebble_dict[dfs_edges[i][1]] > 0)
                        and (dfs_edges[i][1] != dis_edge_list[index][0])
                        and (dfs_edges[i][1] != dis_edge_list[index][1])
                    ):
                        flag = False
                        # print(1,dfs_edges[i])
                        # print(dfs_edges[i])
                        pebble_dict[dfs_edges[i][1]] += -1
                        pebble_dict[dfs_edges[i][0]] += 1
                        direct_edge.remove((dfs_edges[i][0], dfs_edges[i][1]))
                        direct_edge.add((dfs_edges[i][1], dfs_edges[i][0]))
                        G.remove_edge(dfs_edges[i][0], dfs_edges[i][1])
                        G.add_edge(dfs_edges[i][1], dfs_edges[i][0])
                        break
                if flag:
                    dfs_edges = list(nx.dfs_edges(G, source=dis_edge_list[index][1]))
                    for i in range(len(dfs_edges)):
                        if (
                            (pebble_dict[dfs_edges[i][1]] > 0)
                            and (dfs_edges[i][1] != dis_edge_list[index][0])
                            and (dfs_edges[i][1] != dis_edge_list[index][1])
                        ):
                            flag = False
                            # print(2,dfs_edges[i])
                            pebble_dict[dfs_edges[i][1]] += -1
                            pebble_dict[dfs_edges[i][0]] += 1
                            direct_edge.remove((dfs_edges[i][0], dfs_edges[i][1]))
                            direct_edge.add((dfs_edges[i][1], dfs_edges[i][0]))
                            G.remove_edge(dfs_edges[i][0], dfs_edges[i][1])
                            G.add_edge(dfs_edges[i][1], dfs_edges[i][0])
                            break
                if flag:
                    # print("pass")
                    direct_edge = pre_direct_edge
                    pebble_dict = pre_pebble_dict
                    G = pre_G
                    index += 1
    else:
        return False
